## Canadian Fire Weather Index (Unedited)

**Pasted from: (with comments and debugging)**

Wang, Y., Anderson, K.R. and Suddaby, R.M., 2015. *Updated source code for calculating fire danger indices in the Canadian Forest Fire Weather Index System.*


"Daily inputs to the system consist of temperature (°C), relative humidity (%), wind speed (km/h), and precipitation mm) over the past 24 hours"

### The Method

In [5]:
import math

In [6]:
class FWICLASS:
    
    # ********* new function **********
    # Defining some attributes of the FWICLASS
    def __init__(self,temp,rhum,wind,prcp):
        self.h = rhum    # Relative humidity
        self.t = temp    # Temperature
        self.w = wind    # Wind
        self.p = prcp    # Precipitation

    
    
    
    # ********* new function **********    
    # Calculating the fine fuel moisture code (FFMC)
    # mo = FFMC on previous day
    # ffmc0 = FFMC as records begin
    # rf = Effective rain fall for calculating FFMC
    # m = Fine Fuel Moisture Content after drying
    # k1 = Intermediate step in calculation of kw
    # kw = Natural log wetting rate, ln (M)/day 
    def FFMCcalc(self,ffmc0):
        mo = (147.2*(101.0 - ffmc0))/(59.5 + ffmc0) #*Eq. 1*#
        if (self.p > 0.5):
            rf = self.p - 0.5 #*Eq. 2*#
            if(mo > 150.0):
                mo = (mo+42.5*rf*math.exp(-100.0/(251.0-mo))*(1.0 - math.exp(-6.93/rf))) + (.0015*(mo - 150.0)**2)*math.sqrt(rf) #*Eq. 3b*#
            elif mo <= 150.0:
                mo = mo+42.5*rf*math.exp(-100.0/(251.0-mo))*(1.0 - math.exp(- 6.93/rf)) #*Eq. 3a*#
            if(mo > 250.0):
                mo = 250.0 
            
    
        # Fine Fuel equilibrium moisture content(EMC) for drying 
        ed = .942*(self.h**.679) + (11.0*math.exp((self.h-100.0)/10.0))+0.18*(21.1-self.t)*(1.0 - 1.0/math.exp(.1150 * self.h)) #*Eq. 4*#

        # Defining m (Fine Fuel Moisture Content after drying )
        if(mo < ed):
            ew = .618*(self.h**.753) + (10.0*math.exp((self.h-100.0)/10.0)) + .18*(21.1-self.t)*(1.0 - 1.0/math.exp(.115 * self.h)) #*Eq. 5*#
            if(mo <= ew):
                kl = .424*(1.0-((100.0-self.h)/100.0)**1.7)+(.0694*math.sqrt(self.w))*(1.0 - ((100.0 - self.h)/100.0)**8) #*Eq. 7a*#
                kw = kl * (.581 * math.exp(.0365 * self.t)) #*Eq. 7b*#
                m = ew - (ew - mo)/10.0**kw #*Eq. 9*#
            elif mo > ew:
                m = mo

        elif(mo == ed):
            m = mo

        elif mo > ed:
            kl =.424*(1.0-(self.h/100.0)**1.7)+(.0694*math.sqrt(self.w))*(1.0-(self.h/100.0)**8) #*Eq. 6a*#
            kw = kl * (.581*math.exp(.0365*self.t)) #*Eq. 6b*#
            m = ed + (mo-ed)/10.0 ** kw #*Eq. 8*#

        # Calculating ffmc output    
        ffmc = (59.5 * (250.0 -m)) / (147.2 + m) #*Eq. 10*#
        if (ffmc > 101.0):
            ffmc = 101.0
        if (ffmc <= 0.0):
            ffmc = 0.0
        return ffmc
    
    
    
    
    
    # ********* new function **********    
    # Calculating duff moisture code (DMC)
    # el = Effective day length in DMC, monthly (FOR CANADA)
    # rk = Log drying rate in DMC, ln (M)/day
    # t = temperature
    # wmi = Duff Moisture Content from previous day
    # wmr = Duff moisture content after rain
    # pr  = DMC after rain
    # dmc0 = 6.0 (constant)
    # mth = month
    def DMCcalc(self,dmc0,mth):
        el = [6.5,7.5,9.0,12.8,13.9,13.9,12.4,10.9,9.4,8.0,7.0,6.0] # hard coded here for canada
        t = self.t
        if (t < -1.1):
            t = -1.1
        rk = 1.894*(t+1.1) * (100.0-self.h) * (el[mth-1]*0.0001) #*Eqs. 16 and 17*#
        if self.p > 1.5:
            ra= self.p
            rw = 0.92*ra - 1.27   #*Eq. 11*#
            wmi = 20.0 + 280.0/math.exp(0.023*dmc0) #*Eq. 12*#
            if dmc0 <= 33.0:
                b = 100.0 /(0.5 + 0.3*dmc0) #*Eq. 13a*#
            elif dmc0 > 33.0:
                if dmc0 <= 65.0:
                    b = 14.0 - 1.3*math.log(dmc0) #*Eq. 13c*#
                elif dmc0 > 65.0:
                    b = 6.2 * math.log(dmc0) - 17.2 #*Eq. 13b*#
            wmr = wmi + (1000*rw) / (48.77+b*rw)   #*Eq. 14*#
            pr = 43.43 * (5.6348 - math.log(wmr-20.0))  #*Eq. 15*#
        elif self.p <= 1.5:
            pr = dmc0
        if (pr<0.0):
            pr = 0.0
        dmc = pr + rk
        if(dmc<= 1.0):
            dmc = 1.0
        return dmc
    

    
    
    
    # ********* new function **********   
    # Calculating drought code:
    # fl = day length adjustment for drought code
    # t = temp
    # pe = Potential evapotranspiration, units of 0.254 mm water/day 
    # mth = month
    # ra = rainfall
    # rw = effective rainfall for drought code calculation
    # smi = Moisture equivalent of previous day’s DC
    # dr = DC after rain
    # dc0 = input constant (15.0)
    def DCcalc(self,dc0,mth):
        fl = [-1.6, -1.6, -1.6, 0.9, 3.8, 5.8, 6.4, 5.0, 2.4, 0.4, -1.6, -1.6]
        t = self.t
        if(t < -2.8):
            t = -2.8
        pe = (0.36*(t+2.8) + fl[mth-1] )/2    #*Eq. 22*# I think this is a linearisation of the Thornthwaite equation???
        if pe <= 0.0:
            pe = 0.0
        if (self.p > 2.8):
            ra = self.p
            rw = 0.83*ra - 1.27 #*Eq. 18*#
            smi = 800.0 * math.exp(-dc0/400.0) #*Eq. 19*#
            dr = dc0 - 400.0*math.log( 1.0+((3.937*rw)/smi) ) #*Eqs. 20 and 21*#
            if (dr > 0.0):
                dc = dr + pe
        elif self.p <= 2.8:
            dc = dc0 + pe
        return dc

    
    
    
    
    # ********* new function **********
    # Calculating Initial Spread Index (ISI)
    # mo = FFMC on previous day
    # ff = Fine fuel moisture function
    def ISIcalc(self,ffmc):
        mo = 147.2*(101.0-ffmc) / (59.5+ffmc)     #*Eq. 1*#
        ff = 19.115*math.exp(mo*-0.1386) * (1.0+(mo**5.31)/49300000.0)     #*Eq. 25*#
        isi = ff * math.exp(0.05039*self.w)    #*Eq. 26*#
        return isi






    # ********* new function **********
    # Calculating build-up index (BUI)
    # dc = drought code
    # dmc = duff moisute code
    def BUIcalc(self,dmc,dc):
        if dmc <= 0.4*dc:
            bui = (0.8*dc*dmc) / (dmc+0.4*dc)     #*Eq. 27a*#
        else:
            bui = dmc-(1.0-0.8*dc/(dmc+0.4*dc))*(0.92+(0.0114*dmc)**1.7)    #*Eq. 27b*#
        if bui <0.0:
            bui = 0.0
        return bui





    # ********* new function **********
    # Calculating fire weather index (FWI)
    # bb = Intermediate FWI
    def FWIcalc(self,isi,bui):
        if bui <= 80.0:
            bb = 0.1 * isi * (0.626*bui**0.809 + 2.0)        #*Eq. 28a*#
        else:
            bb = 0.1*isi*(1000.0/(25. + 108.64/math.exp(0.023*bui)))        #*Eq. 28b*#
        if(bb <= 1.0):
            fwi = bb        #*Eq. 30b*#
        else:
            fwi = math.exp(2.72 * (0.434*math.log(bb))**0.647)        #*Eq. 30a*#
        return fwi

In [7]:
def main():
    ffmc0 = 85.0
    dmc0 = 6.0
    dc0 = 15.0
    infile = open('data.txt','r') #space gapped text file
    outfile = open('fwioutput.txt','w')
    
    try:
        for line in infile:
            mth,day,temp,rhum,wind,prcp=[float(field) for field in line.strip().lstrip('[').rstrip(']').split()]
            if rhum > 100.0:
                rhum = 100.0
            mth = int(mth)
            print(temp,rhum,wind,prcp)
            fwisystem = FWICLASS(temp,rhum,wind,prcp)
            ffmc = round(fwisystem.FFMCcalc(ffmc0),70)      # Rounding to 1dp to check if this matches their results?
            dmc  = round(fwisystem.DMCcalc(dmc0,mth),70)
            dc   = round(fwisystem.DCcalc(dc0,mth),70)
            isi  = round(fwisystem.ISIcalc(ffmc),70)
            bui  = round(fwisystem.BUIcalc(dmc,dc),70) 
            fwi  = round(fwisystem.FWIcalc(isi,bui),70)
            ffmc0 = ffmc
            dmc0 = dmc
            dc0 = dc
            outfile.write('%s %s %s %s %s %s %s %s %s %s %s %s\n'%(str(mth),str(day),str(temp),str(rhum),str(wind),str(prcp),str(round(ffmc,1)),str(round(dmc,1)),str(round(dc,1)),str(round(isi,1)),str(round(bui,1)),str(round(fwi,1))))
    finally:
        infile.close()
        outfile.close()


### Testing

In [8]:
import numpy as np
import pandas as pd
df = pd.read_csv('test_data.txt', sep=" ")
df = df.drop(['FFMC','DMC','DC','ISI','BUI','FWI'],axis=1)
df.head()
np.savetxt(r'data.txt', df.values, fmt='%f')
print('Input Data:')
df.head(8)

Input Data:


,Month,Day,Temp.,RH,Wind,Rain
0,4,13,17.0,42.0,25.0,0.0
1,4,14,20.0,21.0,25.0,2.4
2,4,15,8.5,40.0,17.0,0.0
3,4,16,6.5,25.0,6.0,0.0
4,4,17,13.0,34.0,24.0,0.0
5,4,18,6.0,40.0,22.0,0.4
6,4,19,5.5,52.0,6.0,0.0
7,4,20,8.5,46.0,16.0,0.0


In [9]:
main()

17.0 42.0 25.0 0.0
20.0 21.0 25.0 2.4
8.5 40.0 17.0 0.0
6.5 25.0 6.0 0.0
13.0 34.0 24.0 0.0
6.0 40.0 22.0 0.4
5.5 52.0 6.0 0.0
8.5 46.0 16.0 0.0
9.5 54.0 20.0 0.0
7.0 93.0 14.0 9.0
6.5 71.0 17.0 1.0
6.0 59.0 17.0 0.0
13.0 52.0 4.0 0.0
15.5 40.0 11.0 0.0
23.0 25.0 9.0 0.0
19.0 46.0 16.0 0.0
18.0 41.0 20.0 0.0
14.5 51.0 16.0 0.0
14.5 69.0 11.0 0.0
15.5 42.0 8.0 0.0
21.0 37.0 8.0 0.0
23.0 32.0 16.0 0.0
23.0 32.0 14.0 0.0
27.0 33.0 12.0 0.0
28.0 17.0 27.0 0.0
23.5 54.0 20.0 0.0
16.0 50.0 22.0 12.2
11.5 58.0 20.0 0.0
16.0 54.0 16.0 0.0
21.5 37.0 9.0 0.0
14.0 61.0 22.0 0.2
15.0 30.0 27.0 0.0
20.0 23.0 11.0 0.0
14.0 95.0 3.0 16.4
20.0 53.0 4.0 2.8
19.5 30.0 16.0 0.0
25.5 51.0 20.0 6.0
10.0 38.0 24.0 0.0
19.0 27.0 16.0 0.0
26.0 46.0 11.0 4.2
30.0 38.0 22.0 0.0
25.5 67.0 19.0 12.6
12.0 53.0 28.0 11.8
21.0 38.0 8.0 0.0
13.0 70.0 20.0 3.8
9.0 78.0 24.0 1.4
11.0 54.0 16.0 0.0
15.5 39.0 9.0 0.0
18.0 36.0 5.0 0.0


In [10]:
df1 = pd.read_csv('test_data.txt', sep=" ")
df2 = pd.read_csv('fwioutput.txt', sep=" ", names=['Month','Day','Temp.','RH','Wind','Rain','FFMC','DMC','DC','ISI','BUI','FWI'])

In [11]:
df2.head()

,Month,Day,Temp.,RH,Wind,Rain,FFMC,DMC,DC,ISI,BUI,FWI
0,4,13.0,17.0,42.0,25.0,0.0,87.7,8.5,19.0,10.9,8.5,10.1
1,4,14.0,20.0,21.0,25.0,2.4,86.2,10.4,23.6,8.8,10.4,9.3
2,4,15.0,8.5,40.0,17.0,0.0,87.0,11.8,26.1,6.5,11.7,7.6
3,4,16.0,6.5,25.0,6.0,0.0,88.8,13.2,28.2,4.9,13.1,6.2
4,4,17.0,13.0,34.0,24.0,0.0,89.1,15.4,31.5,12.6,15.3,14.8


In [12]:
df1.head()

,Month,Day,Temp.,RH,Wind,Rain,FFMC,DMC,DC,ISI,BUI,FWI
0,4,13,17.0,42.0,25.0,0.0,87.7,8.5,19.0,10.9,8.5,10.1
1,4,14,20.0,21.0,25.0,2.4,86.2,10.4,23.6,8.8,10.4,9.3
2,4,15,8.5,40.0,17.0,0.0,87.0,11.8,26.1,6.5,11.7,7.6
3,4,16,6.5,25.0,6.0,0.0,88.8,13.2,28.2,4.9,13.1,6.2
4,4,17,13.0,34.0,24.0,0.0,89.1,15.4,31.5,12.6,15.3,14.8


In [13]:
print(df2.sub(df1).head(49)) #checking against published data

    Month  Day  Temp.   RH  Wind  Rain  FFMC  DMC   DC  ISI  BUI  FWI
0       0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
1       0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
2       0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
3       0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
4       0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
5       0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
6       0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
7       0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
8       0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
9       0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
10      0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
11      0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
12      0  0.0    0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
13      0  0.0    0.